### GBM(Gradient Boost Model)로 삼성전자 2주 후 주가 예측하기

- 생성일: 2024.04.11
- 파이썬 버전: 3.11.5

In [1]:
# data
import pandas as pd
import numpy as np

# finance
import financeDataLoader as fdl # 사용자 정의 함수
import financeFuncs as ffs # 사용자 정의 함수

# Model
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split

# visualize
import matplotlib.pyplot as plt

# tunning
import optuna


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tickers = ['005930.KS', '^KS11']
start_date = '2000-01-01' # 날짜 설정
tick_df = fdl.tickDataload(tickers, start_date=start_date)
tick_df['SAMSUNG'] = tick_df.pop('005930.KS')
print(tick_df.keys())

[*********************100%%**********************]  2 of 2 completed
dict_keys(['KOSPI', 'SAMSUNG'])


In [3]:
tick_df['SAMSUNG']['Volume'].replace(0, pd.NA, inplace=True)

tick_df['SAMSUNG'].fillna(method='ffill', inplace=True)

tick_df['KOSPI']['Volume'].replace(0, pd.NA, inplace=True)

tick_df['KOSPI'].fillna(method='ffill', inplace=True)

/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_8974/2248947292.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tick_df['SAMSUNG'].fillna(method='ffill', inplace=True)
/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_8974/2248947292.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tick_df['KOSPI'].fillna(method='ffill', inplace=True)


In [4]:
# # SAMSUNG 데이터프레임에서 모든 열에 NA 값이 있는 행을 삭제
# tick_df['SAMSUNG'].dropna(how='any', inplace=True)

# # KOSPI 데이터프레임에서 모든 열에 NA 값이 있는 행을 삭제
# tick_df['KOSPI'].dropna(how='any', inplace=True)


In [5]:
# tick_df['SAMSUNG']['Volume'].replace(0, pd.NA, inplace=True)

# tick_df['SAMSUNG']['Volume'].fillna(method='ffill', inplace=True)

# tick_df['KOSPI']['Volume'].replace(0, pd.NA, inplace=True)

# tick_df['KOSPI']['Volume'].fillna(method='ffill', inplace=True)

In [6]:
bm_df = tick_df['KOSPI'] # benchmark

# update tick_df
for ticker, data in tick_df.items():
    tick_df[ticker] = ffs.feature_engineering(data, bm_df, n_days=10) # 종속변수: 2주(10일) 후

In [7]:
# 2010년 이후 데이터만 사용
cut_date = '2010-01-01'
tick_df['SAMSUNG'] = tick_df['SAMSUNG'][tick_df['SAMSUNG'].index >= cut_date]

### train, test split

In [8]:
# X, y 분할
target = "price_diff"
X = tick_df['SAMSUNG'].drop(columns=target)
y = tick_df['SAMSUNG'][target]

In [9]:
# 데이터 분할
split_cond = int(len(y) * 0.8)
X_train, X_test, y_train, y_test = X[:split_cond], X[split_cond:], y[:split_cond], y[split_cond:]

### Model

In [10]:
# GBM 회귀 모델 초기화
lgbm_regressor = LGBMRegressor(random_state=42)

In [11]:
# 모델 훈련
lgbm_regressor.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000758 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9798
[LightGBM] [Info] Number of data points in the train set: 2837, number of used features: 39
[LightGBM] [Info] Start training from score 0.004161


LGBMRegressor(random_state=42)

In [12]:
# 예측
predictions = lgbm_regressor.predict(X_test)

In [13]:
# 성능 평가
#mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.016888158873430696


### MSE 값이 매우 낮다.
종속변수의 값이 어딘가 포함되어 있을 것이다.

In [14]:
# import numpy as np
# from sklearn.model_selection import KFold

# def custom_split(X, y, n_splits, n_start, n_end):
#     kf = KFold(n_splits=n_splits)
#     splits = []

#     for train_index, test_index in kf.split(X):
#         # Exclude n_start and n_end samples from the start and end of the test data
#         if len(test_index) > (n_start + n_end):
#             test_index = test_index[n_start:-n_end]
#         elif len(test_index) > n_start:
#             test_index = test_index[n_start:] if train_index[-1] < test_index[0] else test_index[:-n_end]
#         elif len(test_index) > n_end:
#             test_index = test_index[:-n_end]

#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]

#         splits.append((X_train, X_test, y_train, y_test))
    
#     return splits


In [15]:
import numpy as np
from sklearn.model_selection import KFold

def custom_split(X, y, n_splits, n_start, n_end):
    kf = KFold(n_splits=n_splits)
    splits = []

    for train_index, test_index in kf.split(X):
        # Exclude n_start and n_end samples from the start and end of the test data
        if len(test_index) > (n_start + n_end):
            test_index = test_index[n_start:-n_end]
        elif len(test_index) > n_start:
            test_index = test_index[n_start:] if train_index[-1] < test_index[0] else test_index[:-n_end]
        elif len(test_index) > n_end:
            test_index = test_index[:-n_end]

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        splits.append((X_train, X_test, y_train, y_test))
    
    return splits


In [16]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import numpy as np
import optuna

def lgb_objective(trial, X, y, n_splits, n_start, n_end):
    # Hyperparameters
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 30, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': 0.03,  # Fixed learning rate
        'n_estimators': 10000,  # Fixed n_estimators
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'objective': trial.suggest_categorical('objective', ['l1', 'l2']),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),  # Added parameter
        'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0),  # Added parameter
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),  # Added parameter
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),  # Added parameter
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),  # Added parameter
        'metric': 'mae',  # Using MAE for validation
        'verbose': -1
    }

    # Custom splits
    splits = custom_split(X, y, n_splits, n_start, n_end)
    
    scores = []

    for X_train, X_test, y_train, y_test in splits:
        # Create LightGBM datasets
        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_test, label=y_test, reference=dtrain)

        # Define callbacks
        callbacks = [lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(0)]

        # Train the model
        model = lgb.train(params, dtrain, valid_sets=[dvalid], valid_names=['valid'], callbacks=callbacks)

        # Predict and calculate the score
        preds = model.predict(X_test, num_iteration=model.best_iteration)
        score = mean_absolute_error(y_test, preds)
        scores.append(score)

    # Return the average score
    average_score = np.mean(scores)
    return average_score


In [17]:
# def lgb_objective(trial, X, y, n_splits, n_start, n_end):
#     # 하이퍼파라미터
#     params = {
#         'num_leaves': trial.suggest_int('num_leaves', 30, 300),
#         'max_depth': trial.suggest_int('max_depth', 3, 12),
#         'learning_rate': 0.01,  # 고정된 learning rate
#         'n_estimators': 10000,  # 고정된 n_estimators
#         'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
#         'subsample': trial.suggest_float('subsample', 0.4, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'objective': trial.suggest_categorical('objective', ['l1', 'l2']),
#         'min_data_in_leaf': trial.suggest_int('min_child_samples', 20, 200),  # 추가된 파라미터
#         'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0),  # 추가된 파라미터
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),  # 추가된 파라미터
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),  # 추가된 파라미터
#         'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),  # 추가된 파라미터
#         'metric': 'mae',  # MAE로 모델 검증
#         'verbose': -1
#     }

#     # Custom splits
#     splits = custom_split(X, y, n_splits, n_start, n_end)

#     # 교차 검증 점수 저장을 위한 리스트
#     scores = []

#     # TimeSeriesSplit에서 제공된 인덱스를 사용하여 교차 검증 실행
#     for X_train, X_test, y_train, y_test in splits:
#             # Create LightGBM datasets
#             dtrain = lgb.Dataset(X_train, label=y_train)
#             dvalid = lgb.Dataset(X_test, label=y_test)

#             # Train the model
#             model = lgb.train(params, dtrain, valid_sets=[dvalid], early_stopping_rounds=100, verbose_eval=False)

#             # Predict and calculate the score
#             preds = model.predict(X_test)
#             score = mean_absolute_error(y_test, preds)
#             scores.append(score)

#     # Return the average score
#     average_score = np.mean(scores)
#     return average_score


In [18]:
import optuna

# Set parameters for custom split
n_splits = 5
n_start = 2
n_end = 2

# Define the objective function for Optuna
def objective(trial):
    return lgb_objective(trial, X, y, n_splits, n_start, n_end)

# Create Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best parameters and score
print(f"Best trial: {study.best_trial.params}")
print(f"Best score: {study.best_trial.value}")

[I 2024-05-25 10:39:04,058] A new study created in memory with name: no-name-fe14284c-e6d2-404b-8b0d-b8c4e630317c


Training until validation scores don't improve for 100 rounds


/opt/homebrew/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[383]	valid's l1: 0.0221202
Training until validation scores don't improve for 100 rounds


/opt/homebrew/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

Early stopping, best iteration is:
[449]	valid's l1: 0.0223655
Training until validation scores don't improve for 100 rounds


/opt/homebrew/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[745]	valid's l1: 0.0179302
Training until validation scores don't improve for 100 rounds


/opt/homebrew/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2024-05-25 10:54:35,836] Trial 99 finished with value: 0.01983922294691819 and parameters: {'num_leaves': 180, 'max_depth': 11, 'min_child_samples': 150, 'subsample': 0.6175823246252384, 'colsample_bytree': 0.7307549679406, 'objective': 'l1', 'min_data_in_leaf': 45, 'min_child_weight': 5.802661197466827, 'reg_alpha': 0.5715658130119764, 'reg_lambda': 0.08354858910834202, 'min_gain_to_split': 0.7334621865895071}. Best is trial 14 with value: 0.01939059788689235.


Early stopping, best iteration is:
[1259]	valid's l1: 0.0165672
Best trial: {'num_leaves': 97, 'max_depth': 8, 'min_child_samples': 142, 'subsample': 0.6388722200733261, 'colsample_bytree': 0.8879305343205433, 'objective': 'l1', 'min_data_in_leaf': 21, 'min_child_weight': 1.8900951984551684, 'reg_alpha': 0.3056431120262518, 'reg_lambda': 0.30518366085134546, 'min_gain_to_split': 0.7843994941128859}
Best score: 0.01939059788689235


In [19]:
# GBM 회귀 모델 Best parameters로 초기화
lgbm_regressor = LGBMRegressor(random_state=42, **study.best_trial.params)

#study.best_trial.params

# 모델 훈련
lgbm_regressor.fit(X_train, y_train)

[LightGBM] [Warning] min_gain_to_split is set=0.7843994941128859, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.7843994941128859
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=142 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] min_gain_to_split is set=0.7843994941128859, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.7843994941128859
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=142 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9798
[LightGBM] [Info] Number of data points in the train set: 2837, number of used features: 39
[LightGBM] [Info] Start training from score 0.002709
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

LGBMRegressor(colsample_bytree=0.8879305343205433, max_depth=8,
              min_child_samples=142, min_child_weight=1.8900951984551684,
              min_data_in_leaf=21, min_gain_to_split=0.7843994941128859,
              num_leaves=97, objective='l1', random_state=42,
              reg_alpha=0.3056431120262518, reg_lambda=0.30518366085134546,
              subsample=0.6388722200733261)

In [20]:
# 예측
predictions = lgbm_regressor.predict(X_test)

[LightGBM] [Warning] min_gain_to_split is set=0.7843994941128859, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.7843994941128859
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=142 will be ignored. Current value: min_data_in_leaf=21


In [21]:
# 성능 평가 -> MSE 값이 하락한 것을 확인할 수 있음.
#mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Squared Error: {mae}")

Mean Squared Error: 0.016500622838679004
